<a href="https://colab.research.google.com/github/praveentn/MidcurveNN/blob/master/prepare_data_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare data

In [1]:
# mount google drive & set working directory
# requires auth (click on url & copy token into text box when prompted)
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

import os
print(os.getcwd())

os.chdir('/content/gdrive/My Drive/Colab Notebooks/MidcurveNN')
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content
/content/gdrive/My Drive/Colab Notebooks/MidcurveNN


In [2]:
!pip install drawSVG

     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 71kB 22.9MB/s 
  Created wheel for drawSVG: filename=drawSvg-1.2.2-cp36-none-any.whl size=19889 sha256=9cb2175938fd5979d53220c38763bb7bc6b0dc2ecf078edcf72b357c93788cdb
  Stored in directory: /root/.cache/pip/wheels/f7/d7/bc/abef999ecd24a56605fe1dcad487857a08fb2fcc90a1ca60ec
  Created wheel for cairocffi: filename=cairocffi-1.0.2-cp36-none-any.whl size=88348 sha256=88b79f7cd6fe30260fc37f0cd1391664c51b646b46849aa092ab254414f0aa23
  Stored in directory: /root/.cache/pip/wheels/e7/5d/6f/fc3c2364dfd3c4cfd15d786b156077c52209d9af45496fdf12
Successfully built drawSVG cairocffi


In [3]:
"""
    Prepare Data: populating input images from raw profile data
    Takes raw data from "data/raw/*" files for both, profile shape (shape.dat) as well as midcurve shape (shape.mid)
    Generates raster image files from svg (simple vector graphics)
    Multiple variations are populated using image transformations.
    These images become input for further modeling (stored in "data/input/*")
"""
import os
import sys
import PIL
import json
import shutil
import numpy as np
import PIL.ImageOps
from random import shuffle
from keras.preprocessing.image import img_to_array, load_img, array_to_img
np.set_printoptions(threshold=sys.maxsize)

from PIL import Image


Using TensorFlow backend.


In [4]:
# working directory
wdir = os.getcwd()
wdir

'/content/gdrive/My Drive/Colab Notebooks/MidcurveNN'

In [0]:
imdim = 100

In [0]:
#input_data_folder = wdir + "\\data\\sample"
#input_data_folder = wdir + "/data/newinput"
#print("input data dir: ", input_data_folder)

In [0]:
raw_data_folder = "data/new_shapes"
input_data_folder = "data/new_images"
pix2pix_data_folder = "/data/pix2pix/datasets/pix2pix"

In [0]:
def read_dat_files(datafolder=raw_data_folder):
    profiles_dict_list = []
    for file in os.listdir(datafolder):
        if os.path.isdir(os.path.join(datafolder, file)):
            continue
        filename = file.split(".")[0]
        profile_dict = get_profile_dict(filename,profiles_dict_list)        
        if file.endswith(".dat"):
            with open(os.path.join(datafolder, file)) as f:
                profile_dict['Profile'] = [tuple(map(float, i.split('\t'))) for i in f]  
        if file.endswith(".mid"):
            with open(os.path.join(datafolder, file)) as f:
                profile_dict['Midcurve'] = [tuple(map(float, i.split('\t'))) for i in f]
                                
        profiles_dict_list.append(profile_dict)
    return profiles_dict_list
  

def get_profile_dict(shapename,profiles_dict_list):
    for i in profiles_dict_list:
        if i['ShapeName'] == shapename:
            return i
    profile_dict = {}
    profile_dict['ShapeName'] = shapename
    return profile_dict


import drawSvg as draw

def create_image_file(fieldname,profile_dict,datafolder=input_data_folder,imgsize=imdim, isOpenClose=True):
    d = draw.Drawing(imgsize, imgsize, origin='center')
    profilepoints = []
    for tpl in profile_dict[fieldname]:
        profilepoints.append(tpl[0])
        profilepoints.append(tpl[1])
    d.append(draw.Lines(profilepoints[0],profilepoints[1],*profilepoints,close=isOpenClose,fill='none',stroke='black'))
    
    shape = profile_dict['ShapeName']
#     d.saveSvg(datafolder+"/"+shape+'.svg')
#    d.savePng(datafolder+"/"+shape+'_'+fieldname+'.png')
    d.savePng(datafolder+"/"+shape+'_'+fieldname+'.png')


def get_original_png_files(datafolder=input_data_folder):
    pngfilenames = []
    for file in os.listdir(datafolder):
        fullpath = os.path.join(datafolder, file)
        if os.path.isdir(fullpath):
            continue
        if file.endswith(".png") and file.find("_rotated_") == -1 and file.find("_translated_")==-1 and file.find("_mirrored_")==-1:
            pngfilenames.append(fullpath)
    return pngfilenames


def mirror_images(pngfilenames, mode=PIL.Image.TRANSPOSE):
    mirrored_filenames = []
    for fullpath in pngfilenames:
        picture= Image.open(fullpath)
        newfilename = fullpath.replace(".png", "_mirrored_"+str(mode)+".png")
        picture.transpose(mode).save(newfilename)
        mirrored_filenames.append(newfilename)
    return mirrored_filenames


def rotate_images(pngfilenames, angle=90):
    for fullpath in pngfilenames:
        picture= Image.open(fullpath)
        newfilename = fullpath.replace(".png", "_rotated_"+str(angle)+".png")
        picture.rotate(angle).save(newfilename)


def translate_images(pngfilenames, dx=1,dy=1):
    for fullpath in pngfilenames:
        picture= Image.open(fullpath)
        x_shift = dx
        y_shift = dy
        a = 1
        b = 0
        c = x_shift #left/right (i.e. 5/-5)
        d = 0
        e = 1
        f = y_shift #up/down (i.e. 5/-5)
        translate = picture.transform(picture.size, Image.AFFINE, (a, b, c, d, e, f))
#         # Calculate the size after cropping
#         size = (translate.size[0] - x_shift, translate.size[1] - y_shift)
#         # Crop to the desired size
#         translate = translate.transform(size, Image.EXTENT, (0, 0, size[0], size[1]))
        newfilename = fullpath.replace(".png", "_translated_"+str(dx)+"_"+str(dy)+".png")
        translate.save(newfilename)


In [0]:
def generate_images(datafolder=input_data_folder):
    
    if not os.path.exists(datafolder):
        os.makedirs(datafolder)    
    else:    
        for file in os.listdir(datafolder):
            if file.endswith(".png") and (file.find("_rotated_") != -1 or file.find("_translated_") !=-1):
                print("files already present, not generating...")
                return
                
    print("transformed files not present, generating...")
    profiles_dict_list = read_dat_files()
    
    print(profiles_dict_list)
    
    for profile_dict in profiles_dict_list:
        create_image_file('Profile',profile_dict,datafolder,imdim,True)
        create_image_file('Midcurve',profile_dict,datafolder,imdim,False)
        
    pngfilenames = get_original_png_files(datafolder)
    mirrored_filenames_left_right = mirror_images(pngfilenames, PIL.Image.FLIP_LEFT_RIGHT)
    mirrored_filenames_top_bottom = mirror_images(pngfilenames, PIL.Image.FLIP_TOP_BOTTOM)
    mirrored_filenames_transpose = mirror_images(pngfilenames, PIL.Image.TRANSPOSE)
    
    files_list_list = [pngfilenames,mirrored_filenames_left_right,mirrored_filenames_top_bottom,mirrored_filenames_transpose]
    for filelist in files_list_list:
        for angle in range(30,360,30):
            rotate_images(filelist,angle)
            
        for dx in range(5,21,5):
            for dy in range(5,21,5):
                translate_images(filelist,dx,-dy)

In [18]:
generate_images()

transformed files not present, generating...
[{'ShapeName': 'RelY', 'Profile': [(2.0, 12.0), (4.0, 14.0), (8.0, 10.0), (12.0, 14.0), (14.0, 12.0), (10.0, 8.0), (10.0, 4.0), (6.0, 4.0), (6.0, 8.0)], 'Midcurve': [(3.0, 13.0), (8.0, 8.0), (13.0, 13.0), (8.0, 8.0), (8.0, 4.0)]}, {'ShapeName': 'RelY', 'Profile': [(2.0, 12.0), (4.0, 14.0), (8.0, 10.0), (12.0, 14.0), (14.0, 12.0), (10.0, 8.0), (10.0, 4.0), (6.0, 4.0), (6.0, 8.0)], 'Midcurve': [(3.0, 13.0), (8.0, 8.0), (13.0, 13.0), (8.0, 8.0), (8.0, 4.0)]}]


In [0]:
# wait till all images are generated before executing the next cell
break

In [0]:
# move images to appropriate directories
# directory names follows the shape name

import os
import shutil

srcpath = input_data_folder
destpath = input_data_folder

for root, subFolders, files in os.walk(srcpath):
    for file in files:
        #print(file)
        subFolder = os.path.join(destpath, file[:4])
        if not os.path.isdir(subFolder):
            os.makedirs(subFolder)
        try:
            shutil.move(os.path.join(root, file), subFolder)
        except:
            pass

In [21]:
print(wdir)

src_shapes = wdir + "/data/new_shapes/"
src_images = wdir + "/data/new_images/"

dest_shapes = wdir + "/data/shapes/"
dest_images = wdir + "/data/images/"


files = os.listdir(src_shapes)
for f in files:
    shutil.move(src_shapes+f, dest_shapes)
    
files = os.listdir(src_images)
for f in files:
    shutil.move(src_images+f, dest_images)



/content/gdrive/My Drive/Colab Notebooks/MidcurveNN
